In [ ]:
import nest_asyncio
nest_asyncio.apply() 

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters


TOKEN = "8134993455:AAHk-xJEYC5rAMtDVCYAmMG-KYBB8dxetSU"
MODEL_PATH = "exported_model"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
model.to(device)
model.eval()


def detect_phishing(text: str):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    
    Temperature = 2.0
    logits = outputs.logits / Temperature
    
    probs = torch.softmax(logits, dim=1)[0]
    phishing_prob = float(probs[1])
    not_phishing_prob = float(probs[0])
    
    return phishing_prob, not_phishing_prob


async def start(update, context):
    await update.message.reply_text("Hello! Send me a message and I'll check for phishing.")

async def analyze(update, context):
    text = update.message.text
    max_len = 128
    
    if len(text.split()) > max_len:
        warning = f"\n\n*Warning:* The message exceeds {max_len} words. Results may be less accurate."
        
    else:
        warning = ""
    
    phishing_prob, not_phishing_prob = detect_phishing(text)
    
    if phishing_prob >= 0.7:
        verdict = "*Very High Phishing Risk*"
        confidence = phishing_prob
    
    elif phishing_prob >= 0.5:
        verdict = "*Potential Phishing Risk*"
        confidence = phishing_prob
        
    else:
        verdict = "*No Obvious Signs of Phishing*"
        confidence = not_phishing_prob

    
    await update.message.reply_markdown(
        f"*Phishing Detection*\n"
        f"**Verdict:** {verdict}\n"
        f"**Confidence:** {confidence:.2%}\n"
        f"{warning}"
    )

async def help_command(update, context):
    help_message = (
        "**Cool Phishing Detector Bot Help**\n\n"
        "Send or forward any text message to the bot, and it will analyze the content to determine whether it is phishing or not.\n\n"
        "*Verdict*: The desicion of the Bot weather if the message is legitimate or phishing.\n\n"
        "*Confidence*: The confidence of the Bot on the verdict it presented.\n\n"
        "**Commands:**\n\n"
        "/start - Start the bot and receive a welcome message.\n"
        "/help - Display this help message.\n"
        "/status - Get information about the model and its running device.\n"
    )
    await update.message.reply_markdown(help_message)
    
async def handle_media(update, context):
    await update.message.reply_text("I'm a text-based analyzer! Please copy the text from the image or file and send it to me for analysis.")
    
async def status_command(update, context):
    device_status = "GPU" if torch.cuda.is_available() else "CPU"
    await update.message.reply_text(
        f"Model is running on {device_status}."
        f"\nModel path: {MODEL_PATH}"
        f"\nTokenizer: {tokenizer.__class__.__name__}"
        f"\nModel: {model.__class__.__name__}"
    )

async def main():
    app = ApplicationBuilder().token(TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(MessageHandler(filters.PHOTO | filters.ATTACHMENT, handle_media))
    app.add_handler(CommandHandler("status", status_command))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, analyze))
    print("Bot is running…")
    await app.run_polling()


await main()


Bot is running…


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\telegram\ext\_utils\networkloop.py", line 134, in network_retry_loop
    await do_action()
  File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\telegram\ext\_utils\networkloop.py", line 127, in do_action
    action_cb_task.result()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "C:\Users\user\Ap